# Create normalized 2 channel images (nucleus & membrane)

#### This notebook normalized the deconvolved nuclear and membrane (B2M/CD298) channels to create a pseudo-colored image for segmentation and visualization. Code provided here normalized the images from patient P58 but analogous code is used for all pateints.

In [ ]:
import numpy as np
import warnings
import os
import napari
import tifffile
import cv2 as cv
import pandas as pd
from pathlib import Path

## PNG_Setting

In [ ]:
# whether to generate PNG

PNG_GENERATE = True

## Define functions

In [ ]:
# normalize image (to 16-bit)
def image_normalization(image, img_min=0, img_max=65535):
    image = np.float32(image)
    epsilon = 1e-12
    image = (image - np.min(image)) * (img_max - img_min) / ((np.max(image) - np.min(image)) + epsilon) + img_min
    return np.uint16(image)

# set-up napari viewer used to combine channels into single pseudo-colored image
def set_viewer(unset_view, contrast_in = None, colormap=None):
    if contrast_in is None:
        contrast_in = [[0, 65535],[0, 65335]] # only retaining DAPI and CD298/B2M
    if colormap is None:
        colormap = ['green', 'blue'] # define colors to use for each channel
    
    # specify channel names
    unset_view.layers['Image'].name = 'CD298'
    unset_view.layers['Image [1]'].name = 'DAPI'

    unset_view.layers['CD298'].contrast_limits = contrast_in[0]
    unset_view.layers['DAPI'].contrast_limits = contrast_in[1]

    unset_view.layers['CD298'].colormap = colormap[0]
    unset_view.layers['DAPI'].colormap = colormap[1]

In [ ]:
# list of all FOV name extensions for given patient
fovs = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']

## Input directory

In [ ]:
# assign directory
warnings.filterwarnings('ignore')
default_dir = '/data/Zhaolab/1_AMLCosMx/Final_scripts/0_RawImages/15_94003_S3/'
os.chdir(default_dir)
cwd = os.getcwd()

## Normalize by averaging raw and deconvolved DAPI channels

In [ ]:
# Read in deconvolved membrane channel, but for dapi channel, average raw/deconvolved files

for i in range(len(fovs)):
    
    # Read dapi channel from raw files
    
    # define base filename
    FileName = '20220215_094003_S3_C902_P99_N99_F0' + fovs[i]
    
    # read in z-stack of 9 slices, take average over all z's
    z_slices = []

    for j in range(9):

        # read tif files
        z = tifffile.imread(FileName + '_Z00' + str(j+1) + '.TIF')
        z_slices.append(z)

    all_z = np.stack(z_slices, axis=0)

    # z-projection - take the average value of each z layer
    img = np.sum(all_z, axis=0, dtype=int) / all_z.shape[0]
    
    # retain only DAPI
    ch4_raw = img[4, :, :]

    # Read CD298 & DAPI channel from deconvolved files
    
    ch0 = [];
    ch4 = [];

    for j in range(9):

        # read deconvolved tif files
        img0 = tifffile.imread('/data/Zhaolab/AML_OME_TIF_img/Huygens_Processed/P58_R1149_S3_decon/P58_R1149_S3/' + FileName + '.ome_cmle_z00' + str(j) + '_ch00.tif')
        ch0.append(img0)
        img4 = tifffile.imread('/data/Zhaolab/AML_OME_TIF_img/Huygens_Processed/P58_R1149_S3_decon/P58_R1149_S3/' + FileName + '.ome_cmle_z00' + str(j) + '_ch04.tif')
        ch4.append(img4)

    all_ch0 = np.stack(ch0, axis=0)
    all_ch4 = np.stack(ch4, axis=0)

    # z-projection - take the average value of each z layer
    stack_ch0 = np.sum(all_ch0, axis=0, dtype=int) / all_ch0.shape[0]
    stack_ch4 = np.sum(all_ch4, axis=0, dtype=int) / all_ch4.shape[0]
    
    # average raw and deconvolved dapi channels
    combo_ch4 = np.stack([stack_ch4, ch4_raw], axis=0)
    stack_ch4 = np.sum(combo_ch4, axis=0, dtype=int) / combo_ch4.shape[0]
    
    
    # combine 2 channels in one matrix
    img = np.stack((stack_ch0, stack_ch4), axis=0)
    
    
    # find normalization cutoff pixel values for nuclear and membrane channels, normalizing from 5th to 99.9th percentile of pixel intensities
    # for P51 FOV03, there was one very bright spot so upper percentile threshold was set to 99.99, not 99.9
    membrane = img[0, :, :]
    mem_low = np.percentile(membrane.flatten(), 5)
    mem_high = np.percentile(membrane.flatten(), 99.9)
    
    nuclei = img[1, :, :]
    nuc_low = np.percentile(nuclei.flatten(), 5)
    nuc_high = np.percentile(nuclei.flatten(), 99.99)
    
    
    # call set viewer function
    viewer = napari.view_image(img, channel_axis=0, show=False)
    set_viewer(viewer)
    napari.run()
    
    # set contrast and color
    contrast = [[mem_low, mem_high],
            [nuc_low, nuc_high]]
    color = [viewer.layers['CD298'].colormap,
         viewer.layers['DAPI'].colormap]
    num = 0
    
    img = img.astype('float64')

    try:
        del img
    finally:
        if PNG_GENERATE:
            img_tif_raw = img
            viewer_1 = napari.view_image(img_tif_raw, channel_axis=0, show=False)
            set_viewer(viewer_1, contrast, color)
            viewer_1.scale_bar.visible = True
            viewer_1.scale_bar.unit = "pixel(0.18 micron/pixel)"
            blended = np.zeros(viewer_1.layers[0].data.shape + (4,))
            for layer in viewer_1.layers:
                # normalize data by clims
                layer.data[layer.data > layer.contrast_limits[1]] = layer.contrast_limits[1]
                layer.data[layer.data < layer.contrast_limits[0]] = layer.contrast_limits[0]
                normalized_data = (layer.data - layer.contrast_limits[0]) / (
                        layer.contrast_limits[1] - layer.contrast_limits[0])
                normalized_flatdata = normalized_data.flatten()
                normalized_flatdata[normalized_flatdata < 0] = 0
                colormapped_data = layer.colormap.map(normalized_flatdata)
                colormapped_data = colormapped_data.reshape(normalized_data.shape + (4,))
                blended = blended + colormapped_data
            blended[..., 3] = 1  # set alpha channel to 1
            img_png = image_normalization(blended)
            img_png = img_png[:, :, (2, 1, 0)]
            cv.imwrite('/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P58_R1149_S3_Normalized/' + FileName + '_normalized.png', img_png, [cv.IMWRITE_PNG_COMPRESSION, 0])

        num = num + 1
        del img_png, blended, viewer_1
    